# Digital Twin Fairways
This notebook provides an example on how to use the digital twin backend. It runs a simulation based on fairway information system of the Netherlands. You can define sites (with cargo), climate conditions and ships. Ships will transport the goods from A to B.

In [1]:
import datetime

import geojson
import simpy
import time

# library to load the fairway information network
import dtv_backend.fis
# the simpy processes and objects
import dtv_backend.simple

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

## Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
with open('../dtv_backend/tests/test-01/config.json') as f:
    config = geojson.load(f)
config.keys()

dict_keys(['sites', 'fleet', 'climate', 'activities'])

## Simulation environment
Setup an simulation environment with a time that starts today. You can also choose a date in the past or future. Using the date 0 won't work on windows. 

In [3]:
# Initialize an environment with a real time
now = datetime.datetime.now()
initial_time = now.timestamp()
env = simpy.Environment(initial_time=initial_time)
env.epoch = now


## Network
Here we load the digital twin network. The topological fairway network derived from the Dutch Fairway Information System. The data is processed to be topological connected and usable for transport network analysis.  

In [ ]:
url = 'https://storage.googleapis.com/et-data-science/dtv/network_digital_twin_v0.1.yaml'
G = dtv_backend.fis.load_fis_network(url)
env.FG = G

## Connect ports to simulation environment
We have a few different entities defined. The port contains cargo and a crane. The crane can be used to load and unload cargo from ships. The site objects (later to be extended with sluices, stopping areas, etc) are geojson features. 

In [6]:
ports = []
for site in config['sites']:
    port = dtv_backend.simple.Port(env, **site['properties'], **site)
    ports.append(port)

## Connect ships to simulation environment
The ships can also contain cargo. The ships can move over the graph. They are instances of the prototype ships from the Rijkswaterstaat ship dataset. You can have multiple copies of the same ship. All ships can work at the same time. 

In [7]:
ships = []
for ship in config['fleet']:
    ship = dtv_backend.simple.Ship(env, **ship['properties'], **ship)
    ships.append(ship)


## Simulation loop
Here we define the major simulation loop. It will run for a number of steps. It will wait for ships to become available. If a ship is available it will get a new assignment.  The assignment consists of a four step procedure. sail to source, load cargo, move to destination, unload cargo. All the activities that are logged in the corresponding object.

We have three stop conditions:
- Time span (2 weeks)
- Number of iterations (n=100)
- All cargo transported 

In [12]:
class Operator(object):
    """This class represents a fleet operator."""
    def __init__(self, env, ships):
        self.env = env
        self.fleet = simpy.Store(env, capacity=len(ships))
        # create list of tasks, any ship can pick them up
        self.tasks = simpy.Store(env, capacity=len(ships))
        for ship in ships:
            self.fleet.put(ship)

    def send_task(self, task):
        """return send task request"""
        return self.tasks.put(task)

    def get_task(self):
        """return get task request"""
        return self.tasks.get()

    def plan(self, source, destination):
        """A process which prepares tasks."""
        outstanding_work = 0
        while True:
            print('preparing task')
            if source.cargo.level <= 0:
                print('port empty', source.cargo.level)
                break
            if source.cargo.level - outstanding_work <= 0 :
                print('waiting for ships to return')
                # wait six hours for tasks to complete
                yield self.env.timeout(3600 * 6)
                # check again
                continue
            # Time it takes to send an assignment (1 minute)
            yield self.env.timeout(60)
            # check the maximuma load and set out tasks for that maximum load
            max_load = min(
                max(ship.max_load for ship in self.fleet.items),
                source.cargo.level - outstanding_work
            )
            self.send_task({
                "source": source, 
                "destination": destination, 
                "max_load": max_load
            })
            # if we set out a task with max_load, don't put more work than we have so keep track of the outstanding work
            outstanding_work += max_load


        
# Setup and start the simulation
env = simpy.Environment()
ship = ships[0]
operator = Operator(env, ships)
env.process(operator.plan(ports[0], ports[1]))
env.process(ship.work_for(operator))
SIM_DURATION = 3600 * 24
env.run(until=SIM_DURATION)

preparing task
preparing task


AttributeError: module 'dtv_backend' has no attribute 'network'

In [10]:
source, destination = ports[0], ports[1]

def loop():
    max_steps = 100
    fleet = simpy.Store(env=env, capacity=len(ships))
    for ship in ships:
        fleet.put(ship)

    for i in range(max_steps):
        if source.cargo.level <= 0:
            break
        # put all ships to work, if one of them is done, 
        # check conditions
        ship = yield fleet.get()
        yield env.process(ship.load_move_unload(source, destination))
        yield fleet.put(ship)
# Define the work
env.process(loop())

<Process(loop) object at 0x12e9a8040>

In [ ]:
two_weeks = now + datetime.timedelta(days=140)
env.run(until=two_weeks.timestamp())

preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing t

preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing t

waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for

waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for

preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing t

...

waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for

waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for

preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing t

preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing t

preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing task
waiting for ships to return
preparing t

## Postprocessing
After the simulation is complete we can load the activity logs into a data frame and export them to our favorite formats or make charts. 

In [ ]:
import pandas as pd
dfs = [pd.DataFrame(ship.log) for ship in ships]
df = pd.concat(dfs)
df['Name'] = df['Meta'].apply(lambda x: x['ship'].name)
df = df.sort_values(['Timestamp', 'AcitivityID'])


In [ ]:
pd.DataFrame(ports[0].log).head()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(7, 4))

source_port = pd.DataFrame(ports[0].log)
source_port['level'] = source_port['Meta'].apply(lambda x: x['source_level'])
destination_port = pd.DataFrame(ports[1].log)
destination_port['level'] = destination_port['Meta'].apply(lambda x: x['destination_level'])

ax.step(source_port['Timestamp'], source_port['level'], label='source')
ax.step(destination_port['Timestamp'], destination_port['level'], label='destination')
ax.set_ylabel('cargo [tonne]')
ax.set_xlabel('time')

ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))

for ship in ships:
    df = pd.DataFrame(ship.log)
